In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from bs4 import BeautifulSoup
import requests
import pandas as pd
from ar_corrector.corrector import Corrector
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing
import numpy as np
from sklearn.metrics import confusion_matrix,accuracy_score
from nltk.stem.isri import ISRIStemmer
from nltk.corpus import stopwords 
import re

In [ ]:
BookName=[]
authers=[]
deps=[]
page_no=[]
publisher=[]
lang=[]
Links=[]
description_list=[]

In [ ]:
def filtering(word_list):
    stopword_list=stopwords.words('arabic')
    wordsfilter=[]
    st = ISRIStemmer()
    for a in word_list.split():
        if a in stopword_list:
            continue 
        stem = st.stem(a)
        wordsfilter.append(stem)
    return " ".join(wordsfilter)

In [ ]:
def deleteDublicatedNames(string):
        splitedName=[]
        name=string
        if 'كتب' in string.split():
            splitedName=string.split('كتب')
        elif 'الكاتب' in string.split():
            splitedName=string.split('الكاتب')
            #print(splitedName)
        splitedName = [i for i in splitedName if i]
            #print(splitedName)
        if len(splitedName)>0:
            name=splitedName[0]
        return " ".join(name.replace('كتب','').replace('الكاتب','').split())

In [ ]:
def AppendDataRecords(name,text):
    authers.append(deleteDublicatedNames(" ".join((text[0].text[12:]).replace('\n'," ").split())))
    lang.append(" ".join((text[2].text[11:]).split()))
    NoOfPages=(text[3].text[14:])
    numeric_filter=filter(str.isdigit,NoOfPages)
    numeric_string="".join(numeric_filter)
    if(numeric_string==''):
        numeric_string='10'
    page_no.append(int(numeric_string))
    publisher.append(" ".join((text[4].text[11:]).replace('دار','').replace('مؤسسة','').replace('\n'," ").split())) 
    BookName.append(" ".join(((name.replace('كتاب','')).replace('PDF','').replace('pdf','')).replace('\n'," ").split()))

In [ ]:
def no_OfRecords(ReadLink,name,text,desc):
    dep=((text[1].text[13:]).replace( 'تحميل','').replace('كتب','').replace('\n\n','\n'))
    ListOfDep=dep.split('\n')
    for x in ListOfDep:
        if x!='' :
            deps.append(" ".join(x.split()))
            Links.append(ReadLink)
            description_list.append(desc.replace('PDF',''))
            AppendDataRecords(name,text)

In [ ]:
url='https://www.arab-books.com/books/%d9%83%d8%aa%d8%a7%d8%a8-%d8%a7%d9%84%d8%b8%d8%a7%d9%87%d8%b1%d8%a9-%d8%a7%d9%84%d8%ac%d9%85%d8%a7%d9%84%d9%8a%d8%a9-%d9%81%d9%8a-%d8%a7%d9%84%d8%a5%d8%b3%d9%84%d8%a7%d9%85-pdf/'
req=requests.get(url)
content=req.text
soup=BeautifulSoup(content)
description=soup.find("div",attrs={'class':'entry-content entry clearfix'})
c=0
for i in description:
    c+=1
    x=re.sub('<[^>]*>', '', str(i) )
    if c>6 and len(x.split())>25  :
        break;

In [ ]:
url='https://www.arab-books.com'
corr = Corrector()
for i in range(1,201):
    #print('page'+str(i))
    req=requests.get(url+'/page/'+str(i)+'/')
    content=req.text
    soup=BeautifulSoup(content)
    ul=str(soup.findAll('ul', attrs={'class':'posts-items'}))
    ul=BeautifulSoup(ul)
    for a in ul.find_all('a',href=True, attrs={'class':'post-thumb'}):
        name=a.get('aria-label')
        link=a.get('href')
        req=requests.get(link)
        content=req.text
        content=BeautifulSoup(content)
        data=str(content.find_all('div', attrs={'class':'book-info'}))
        data=BeautifulSoup(data)
        parent=data.find('ul')
        if  not parent:
            continue
        text=parent.find_all("li")
        div=str(content.find_all('div',attrs={'class':'read-link-bottom'}))
        div=BeautifulSoup(div)
        link=div.find('a',href=True)
        if (not link):
            ReadLink='Not Avialable'
        else : ReadLink=link.get('href')
        description=content.find("div",attrs={'class':'entry-content entry clearfix'})
        desc=" "
        c=0
        for j in description:
            c+=1
            desc=re.sub('<[^>]*>', '', str(j) )
            if c>6 and len(desc.split())>25 and desc !=x :
                break;
        no_OfRecords(ReadLink,name,text,filtering(corr.contextual_correct((desc.replace('PDF','')))))

In [ ]:
dic={'اسم الكتاب': BookName ,'الكاتب': authers , 'القسم': deps , 'عدد صفحات الكتاب': page_no , 'اللغه' : lang , 'دار النشر':publisher ,'رابط قراءه الكتاب' : Links ,'ملخص الكتاب' : description_list }
df = pd.DataFrame(dic)
df.to_csv('text1.csv',index=False,header=True,encoding='utf-8-sig')

In [3]:
df=pd.read_csv("text (1).csv",encoding= 'utf-8-sig')
df

,اسم الكتاب,الكاتب,القسم,عدد صفحات الكتاب,اللغه,دار النشر,رابط قراءه الكتاب,ملخص الكتاب
0,نظرة جديدة إلى التراث,محمد عمارة,العقيدة,10,عربي,قتيبة,Not Avialable,كتب نظر جدد ترث عبر كتب حمد عمر اهم فكر سلم شه...
1,نظرة جديدة إلى التراث,محمد عمارة,اسلامية,10,عربي,قتيبة,Not Avialable,كتب نظر جدد ترث عبر كتب حمد عمر اهم فكر سلم شه...
2,تحديات لها تاريخ,محمد عمارة,العقيدة,10,عربي,ال العربية,Not Avialable,كتب تحد ارخ عبر كتب حمد عمر اهم فكر سلم شهر أس...
3,تحديات لها تاريخ,محمد عمارة,اسلامية,10,عربي,ال العربية,Not Avialable,كتب تحد ارخ عبر كتب حمد عمر اهم فكر سلم شهر أس...
4,تجديد الدنيا بتجديد الدين,محمد عمارة,العقيدة,10,عربي,النهضة,Not Avialable,كتب جدد دنا جدد دين عبر كتب حمد عمر اهم فكر سل...
...,...,...,...,...,...,...,...,...
5916,عصفور من الشرق,توفيق الحكيم,الأدب العربي,196,العربية,مكتبة مصر,https://www.arab-books.com/books/%d9%83%d8%aa%...,شهر كتب روي قصص دبي كتب عصفور شرق ؤلف وفق حكم ...
5917,عهد الشيطان,توفيق الحكيم,القصص القصيرة,164,العربية,مكتبة مصر,https://www.arab-books.com/books/%d9%83%d8%aa%...,كتب عهد شيط شهر قصص لسف كتب وفق حكم ظهر ذكء ؤل...
5918,في ميزان النقد الاستشراقى,توفيق الحكيم,الثقافة العامة,8,العربية,NaN,https://www.arab-books.com/books/%d9%83%d8%aa%...,شهر كتب نقد كتب ميز نقد استشراقي ؤلف وفق حكم ك...
5919,قالبنا المسرحى,توفيق الحكيم,الثقافة العامة,196,العربية,مكتبة مصر,https://www.arab-books.com/books/%d9%83%d8%aa%...,شهر كتب ثقف عمة كتب بنا سرحى ؤلف وفق حكم كتب ب...


In [4]:
columns=['اسم الكتاب','الكاتب','القسم','ملخص الكتاب']
df=df[columns]

In [5]:
df['رقم القسم'] = df['القسم'].factorize()[0]
df['ملخص الكتاب'].replace("",np.nan,inplace=True)
df.dropna(subset=['ملخص الكتاب'],inplace=True)
df.replace("",np.nan,inplace=True)
df.dropna(inplace=True)
prediction=[]
Accuracy_Score=[]
df

C:\Users\PC\AppData\Local\Temp/ipykernel_11376/520135415.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['رقم القسم'] = df['القسم'].factorize()[0]
C:\Users\PC\anaconda3\lib\site-packages\pandas\core\generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
C:\Users\PC\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

,اسم الكتاب,الكاتب,القسم,ملخص الكتاب,رقم القسم
0,نظرة جديدة إلى التراث,محمد عمارة,العقيدة,كتب نظر جدد ترث عبر كتب حمد عمر اهم فكر سلم شه...,0
1,نظرة جديدة إلى التراث,محمد عمارة,اسلامية,كتب نظر جدد ترث عبر كتب حمد عمر اهم فكر سلم شه...,1
2,تحديات لها تاريخ,محمد عمارة,العقيدة,كتب تحد ارخ عبر كتب حمد عمر اهم فكر سلم شهر أس...,0
3,تحديات لها تاريخ,محمد عمارة,اسلامية,كتب تحد ارخ عبر كتب حمد عمر اهم فكر سلم شهر أس...,1
4,تجديد الدنيا بتجديد الدين,محمد عمارة,العقيدة,كتب جدد دنا جدد دين عبر كتب حمد عمر اهم فكر سل...,0
...,...,...,...,...,...
5916,عصفور من الشرق,توفيق الحكيم,الأدب العربي,شهر كتب روي قصص دبي كتب عصفور شرق ؤلف وفق حكم ...,6
5917,عهد الشيطان,توفيق الحكيم,القصص القصيرة,كتب عهد شيط شهر قصص لسف كتب وفق حكم ظهر ذكء ؤل...,68
5918,في ميزان النقد الاستشراقى,توفيق الحكيم,الثقافة العامة,شهر كتب نقد كتب ميز نقد استشراقي ؤلف وفق حكم ك...,8
5919,قالبنا المسرحى,توفيق الحكيم,الثقافة العامة,شهر كتب ثقف عمة كتب بنا سرحى ؤلف وفق حكم كتب ب...,8


In [ ]:
Max=0
for i in df['رقم القسم']:
    if Max < i :
        Max=i
print(Max)
classes=[ [0]*(Max+1) for _ in range(986) ]

In [ ]:
#for i in range(len(df['رقم القسم'])):
    #classes[i][df['رقم القسم'][i]]=1

In [6]:
count_vect=CountVectorizer()
X_train, X_test, y_train, y_test = train_test_split(df['اسم الكتاب'], df['رقم القسم'], test_size = 0.2, random_state = 0)
X_train_counts=count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.toarray()
nb= MultinomialNB()
nb.fit(X_train_tfidf, y_train)
nb.score(X_train_tfidf, y_train)
X_test_counts=count_vect.transform(X_test)
X_test_tfidf = tfidf_transformer.fit_transform(X_test_counts)
prediction.append(nb.predict(X_test_tfidf))
cm = confusion_matrix(y_test, prediction[0])
Accuracy_Score.append(accuracy_score(y_test, prediction[0]))
print(Accuracy_Score[0])

0.5072595281306715


In [7]:
le = preprocessing.LabelEncoder()
x=[]
y=[]
x.append(le.fit_transform(df.iloc[:,0]))
x=np.transpose(x)

In [9]:
#train and test data
x_train, x_test, y_train, y_test = train_test_split( x, df['رقم القسم'], test_size=0.2,random_state = 0)
gnb = GaussianNB()
#training model
gnb.fit(x_train, y_train)
prediction.append(gnb.predict(x_test))
Accuracy_Score.append(accuracy_score(y_test, prediction[1]))
Accuracy_Score[1]

0.25680580762250454

In [10]:
count_vect=CountVectorizer()
X_train, X_test, y_train, y_test = train_test_split(df['ملخص الكتاب'], df['رقم القسم'], test_size = 0.2, random_state = 0)
X_train_counts=count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.toarray()
nb.fit(X_train_tfidf, y_train)
nb.score(X_train_tfidf, y_train)
X_test_tfidf=count_vect.transform(X_test)
prediction.append(nb.predict(X_test_tfidf))
cm = confusion_matrix(y_test, prediction[1])
Accuracy_Score.append(accuracy_score(y_test, prediction[2]))
Accuracy_Score[2]

0.25680580762250454

In [11]:
Final_pred=[]
Max=-1
indx=0
for i in range(len(Accuracy_Score)):
    if Accuracy_Score[i]>Max:
        Max=Accuracy_Score[i]
        index=i
for i in range(len(prediction[0])):
    if prediction[0][i]==prediction[1][i]:
        Final_pred.append(prediction[0][i])
    elif prediction[0][i]==prediction[2][i]:
        Final_pred.append(prediction[0][i])
    elif prediction[1][i]==prediction[2][i]:
         Final_pred.append(prediction[0][i])
    else : Final_pred.append(prediction[index][i])
print(accuracy_score(y_test, Final_pred))

0.5072595281306715


In [12]:
x=[]
for desc in df['ملخص الكتاب']:
    text=''.join([i for i in desc if not i.isdigit()])
    output = re.sub(r'\s*[A-Za-z]+\b', '' , text)
    x.append(output)
x=pd.DataFrame({'ملخص الكتاب':x})
x

,ملخص الكتاب
0,كتب نظر جدد ترث عبر كتب حمد عمر اهم فكر سلم شه...
1,كتب نظر جدد ترث عبر كتب حمد عمر اهم فكر سلم شه...
2,كتب تحد ارخ عبر كتب حمد عمر اهم فكر سلم شهر أس...
3,كتب تحد ارخ عبر كتب حمد عمر اهم فكر سلم شهر أس...
4,كتب جدد دنا جدد دين عبر كتب حمد عمر اهم فكر سل...
...,...
5501,شهر كتب روي قصص دبي كتب عصفور شرق ؤلف وفق حكم ...
5502,كتب عهد شيط شهر قصص لسف كتب وفق حكم ظهر ذكء ؤل...
5503,شهر كتب نقد كتب ميز نقد استشراقي ؤلف وفق حكم ك...
5504,شهر كتب ثقف عمة كتب بنا سرحى ؤلف وفق حكم كتب ب...


In [13]:
X_train_counts=count_vect.fit_transform(x['ملخص الكتاب'])
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
dataFrame=pd.DataFrame(X_train_tfidf.toarray(),columns=count_vect.get_feature_names())
dataFrame

C:\Users\PC\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,_و,bằ,bệ,chú,chữ,corá,cá,duâ,khuyế,khí,...,ﻳﺴﺘﻌﻤﻠﮫﺎ,ﻳﺼﯿﺮ,ﻳﻀﺎ,ﻳﻌﺮف,ﻳﻘﻈﺘﻪ,ﻷھﻠﻪ,ﻷﺧﻼق,ﻷﻣﺮ,ﻹﻣﺎم,ﻹﻳﺠﺎﺑﻲ
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5501,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5502,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5503,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5504,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
importantWords=[]
for i in dataFrame:
    for j in dataFrame[i]:
        if j>.6:
            importantWords.append([j,i])
            break;
            

In [15]:
print('Important words are:')
importantWords={'TF-IDF':[i[0]for i in importantWords ],'word':[i[1]for i in importantWords ]}
importantWords=pd.DataFrame(importantWords)
importantWords

Important words are:


,TF-IDF,word
0,0.613606,أزق
1,0.664270,أساطير
2,0.605707,أول
3,0.710194,ؤسس
4,0.689394,ابء
...,...,...
256,0.701049,يرل
257,0.635507,يعن
258,0.657148,يقظ
259,0.606409,يهد
